<a href="https://colab.research.google.com/github/jyimz/mbody.github.io/blob/main/InteriorAIPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Let's install the relevant packages
!pip install diffusers
!pip install gradio -q

In [ ]:
# Let's load the libraries
from diffusers import StableDiffusionPipeline
import torch

In [ ]:
# Let's import the models
model_id = "stabilityai/stable-diffusion-2"

pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to("cuda") #we push the model to the GPU

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:270: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
vae/diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Recommendation Model based on Image Upload

In [ ]:
import tensorflow as tf #Imports TensorFlow, a powerful library for numerical computation and machine learning.
import numpy as np
from PIL import Image
import gradio as gr

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import gradio as gr

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Assuming we only identify a few basic scene types for simplicity, each associated with furniture recommendations
#maybe we can incorporate ChatGPT here to give us feedback, currently we are only using the following...
FURNITURE_RECOMMENDATIONS = {
    'living room': ['Add an area rug', 'Create a gallery wall', 'Add a feature wall', 'Add some plants'],
    'bedroom': ['Incorporate mirrors', 'Add mood lighting', 'Buy new bedding'],
    'dining room': ['Change the light fixture', 'Add accent chair', 'Add wall paneling'],
    'office': ['Add standing desk', 'Add laptop stand', 'Include natural light'],
    'kitchen': ['Install under-cabinet lighting for ambiance', 'Upgrade to stainless steel appliances', 'Opt for sleek and handleless cabinets', 'Incorporate a kitchen island with seating'],
    'bathroom': ['Install a rainfall showerhead', 'Use high-quality, moisture-resistant paint', 'Incorporate storage solutions like shelves or cabinets'],

    # Add as many as needed
}

def predict_furniture(image):
    raw_image = Image.fromarray(image.astype('uint8'), 'RGB')
    inputs = processor(raw_image, return_tensors="pt")

    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    # Extract room type from the generated caption
    room_type = None
    for key in FURNITURE_RECOMMENDATIONS.keys():
        if key in caption.lower():
            room_type = key
            break

    if room_type is None:
        room_type = 'unknown: Please contact our customer support'  # If the caption doesn't match any predefined room type

    # Fetch the furniture recommendations based on the determined room type
    recommendations = FURNITURE_RECOMMENDATIONS.get(room_type, [])

    # Format the recommendations as a response
    return 'Recommendations for {}: {}'.format(room_type, ', '.join(recommendations))

iface = gr.Interface(fn=predict_furniture,
                     inputs="image",
                     outputs="text",
                     title="Interior.AI",
                     description="Upload a room image to get furniture addition recommendations")

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://04e99cb9700ac7d585.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Uploading Recommendation to generate a new image

In [ ]:
def text2image(prompt):
  """Generates an image from a text prompt, handling programming-related
     requests with code blocks, if applicable.

  Args:
    prompt: The text description for the desired image.

  Returns:
    PIL.Image: The generated image.
  """

  image = pipe(prompt, height=768, width=768, guidance_scale = 10).images[0]

  image.save("sd_image.png")

  return image

In [ ]:
import gradio as gr

gr.Interface(text2image, gr.Text(), gr.Image(), title = 'Interior.AI').launch(share = True, debug = True)

#add a code when it generates

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1b22e7954a3ad91b29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]